In [66]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez
from Bio import SeqIO

In [43]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [69]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
record = Entrez.read(Entrez.esearch(db = 'nuccore', term = q))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
records = SeqIO.parse(handle, 'gb')

In [70]:
for record in records:
    print record.name

NM_001141980
NM_001141979
NM_005657
NM_001276699
NM_001276695
